In [1]:
import nltk
import spacy
import numpy as np
import pandas as pd
import os
import pickle

In [2]:
import sematch

In [3]:
os.chdir('Training  Data')

In [4]:
train_data = pd.read_csv('train_new.csv')
train_data = train_data.drop(['Unnamed: 0'],axis=1)

In [5]:
len(train_data['id'].unique())

9992

In [6]:
train_data.head()

,id,special,whole,label,masked
0,0,orange juice,he pour orange juice on his cereal,0,he pour [MASK] on his cereal
1,0,milk,he pour milk on his cereal,1,he pour [MASK] on his cereal
2,1,apple,he drink apple,0,he drink [MASK]
3,1,milk,he drink milk,1,he drink [MASK]
4,2,a,jeff run a mile today,1,jeff run [MASK] mile today


# Retrieve entities and concepts

In [7]:
#random.sample(range(1, 100), 3)
query_ids = list(range(75,150))
samples = train_data.loc[train_data['id'].isin(query_ids)]

In [ ]:
key = "6f61e8fa-aa8b-43f6-b2fd-6f35e88ede42"

In [ ]:
import requests
import json

# An API Error Exception
class APIError(Exception):
    def __init__(self, status):
        self.status = status
    def __str__(self):
        return "APIError: status={}".format(self.status)

def query(text):
    # Base URL for Spotlight API
    base_url = "https://babelfy.io/v1/disambiguate?"
    # Parameters 
    # 'text' - text to be annotated 
    # 'confidence' -   confidence score for linking
    params = {"text": text, "lang": 'en', "posTag":"STANDARD",
              "annType":"ALL", "key":key}
    # Response content type
    headers = {'accept': 'text/html'}
    # GET Request
    res = requests.get(base_url, params=params, headers=headers)
    if res.status_code != 200:
        # Something went wrong
        raise APIError(res.status_code)
    return json.loads(res.text)

def query_wn_synsets(word):
    base_url = 'https://babelnet.io/v5/getSynset'
    params = {"id": word['babelSynsetID'], "targetLang": 'en', "key":key}
    # Response content type
    headers = {'accept': 'text/html'}
    # GET Request
    res = requests.get(base_url, params=params, headers=headers)
    if res.status_code != 200:
        # Something went wrong
        raise APIError(res.status_code)
    bn_synset = json.loads(res.text)
    return bn_synset

In [ ]:
cached_res = {}
cached_df = samples.groupby(['id','label']).first().reset_index()

In [13]:
for i, row in cached_df.iterrows():
    queryid = row['id']
    query_label = row['label']
    text = row['whole']
    print(text)
    query_res = query(text)
    cached_res[(queryid, query_label)] = query_res

you will never see a teacher inside of a school
you will find teacher in a school
she cut a pizza with an advertising board
she cut a pizza with a knife
they build the sun
they build a house
the man put some building in the microwave
the man put some muffin in the microwave
there wa a chicken flying
there wa a bird flying
he play the cow very well
he play the harp very well
you are still alive if you do not have a heartbeat
if your heartbeat stop then you are class a dead
the trampoline jump on him
he jump on the trampoline
georgia tech train dragon
georgia tech be a school
she wa going to school in a lunch box
she wa going to school in a bus
the moon get fix and that how it return to full
the moon turn full about once a month
the man pay the bill with water
the man pay the bill with cash
you are cooking the music
you are cooking the dinner
she write the story in water
she write the story in paper
the storage in every computer be unlimited
a computer can run out of storage
people only 

In [ ]:
len(cached_res)

In [51]:
cached_spe_res = {}
cached_spe_df = samples.groupby(['id','label']).first().reset_index()

In [52]:
for i, row in cached_spe_df.iterrows():
    queryid = row['id']
    query_label = row['label']
    text = row['special']
    print(text)
    query_res = query(text)
    cached_spe_res[(queryid, query_label)] = query_res

never see a
find
an advertising board
a knife
the sun
a house
building
muffin
chicken
bird
cow
harp
still alive if you do not have
if your heartbeat stop then
jump on him
he jump on
train dragon
be a school
lunch box
bus
get fix and that how it return to
turn
water
cash
music
dinner
water
paper
the
a computer can run out of
only
sad
hamburger
coffee
pillow
stone
the
a
teeth
beard
star
corndog
ghillie suit
shoe
fridge
music
be not allow to
can
are comfortable
cause discomfort
low
a diet be a great way to lower
computer
carbohydrate
electronic good
non veg food
europe
france
the moon
london
the
when a
sheep
lady
can not get
drinking alcohol get
fire hydrant
gun
snow
tree
the build
he build a
singing
growing
fridge
knife
sky
barn
no one us pen to
people
a bathtub
the ocean
would never
be likely to throw a
sulfuric acid
water
tire
tea
screwdriver
remote
giraffe
whale
a city to
to a new city
closet
freezer
people be
plane wa
hand
bread
curb
trash out
rock
candle
fly on
deliver
perform a sur

In [53]:
len(cached_spe_res)

150

In [16]:
cached_synsets = {}
for id_label, words in cached_res.items():
    cached_synsets[id_label] = [query_wn_synsets(word) for word in words]

In [54]:
cached_spe_synsets = {}
for id_label, words in cached_spe_res.items():
    cached_spe_synsets[id_label] = [query_wn_synsets(word) for word in words]

In [18]:
with open('cached-synset-75-150.pickle','wb') as handle:
    pickle.dump(cached_synsets, handle)

In [55]:
with open('cached-spe-synset-75-150.pickle','wb') as handle:
    pickle.dump(cached_spe_synsets, handle)

# Analyze wordnet similarity

In [56]:
with open('cached-synset-75-150.pickle','rb') as handle:
    cached_synsets = pickle.load(handle)
with open('cached-spe-synset-75-150.pickle','rb') as handle:
    cached_spe_synsets = pickle.load(handle)

In [57]:
from nltk.corpus import wordnet as wn

In [58]:
cached_synsets[(75,0)]

[{'senses': [{'type': 'WordNetSense',
    'properties': {'wordNetSenseNumber': 1,
     'wordNetOffset': '01825237v',
     'wordNetSynsetPosition': 1,
     'fullLemma': 'desire',
     'simpleLemma': 'desire',
     'source': 'WN',
     'senseKey': 'desire%2:37:00::',
     'frequency': 0,
     'language': 'EN',
     'pos': 'VERB',
     'synsetID': {'id': 'bn:00086682v', 'pos': 'VERB', 'source': 'BABELNET'},
     'translationInfo': '',
     'pronunciations': {'audios': [{'lemma': 'desire',
        'language': 'EN',
        'filename': 'en-us-desire.ogg'}],
      'transcriptions': ['[/dɪˈzaɪɹ/]', '[/dɪˈzaɪə/]']},
     'bKeySense': False,
     'idSense': 98071902}},
   {'type': 'WordNetSense',
    'properties': {'wordNetSenseNumber': 1,
     'wordNetOffset': '01825237v',
     'wordNetSynsetPosition': 2,
     'fullLemma': 'want',
     'simpleLemma': 'want',
     'source': 'WN',
     'senseKey': 'want%2:37:00::',
     'frequency': 0,
     'language': 'EN',
     'pos': 'VERB',
     'synsetID': 

In [67]:
wn_concepts = {(idx, label):set() for (idx, label) in cached_synsets.keys()}
for (idx, label), all_word_synsets in cached_synsets.items(): # for each sentence
    for single_word_synsets in all_word_synsets: # for each synset of word
        # only keep the first wordnet words
        for synset in single_word_synsets['senses']:
            if synset['type'] == 'WordNetSense':
                sense_key = synset['properties']['senseKey']
                #lemma = wn.lemma_from_key(sense_key)
                #synsetkey = wn.synset(lemma)
                try:
                    synsetkey = wn.synset_from_sense_key(sense_key)
                    #print(synsetkey)
                    wn_concepts[(idx, label)].add(synsetkey)
                except Exception as e:
                    print(e)
                    print(sense_key,'wrong wordnet error')
                

lemma 'well' with part of speech 'r' has only 13 senses
well%4:02:15:: wrong wordnet error
lemma 'well' with part of speech 'r' has only 13 senses
well%4:02:15:: wrong wordnet error
lemma 'go_away' with part of speech 'v' has only 4 senses
go_away%2:38:05:: wrong wordnet error
lemma 'go_away' with part of speech 'v' has only 4 senses
go_away%2:38:05:: wrong wordnet error
adjective satellite requested but only plain adjective found for lemma 'inexhaustible'
inexhaustible%5:00:00:infinite:00 wrong wordnet error
lemma 'modify' with part of speech 'v' has only 3 senses
modify%2:30:10:: wrong wordnet error
lemma 'modify' with part of speech 'v' has only 3 senses
modify%2:30:10:: wrong wordnet error
lemma 'happen' with part of speech 'v' has only 5 senses
happen%2:40:12:: wrong wordnet error
lemma 'chance' with part of speech 'v' has only 3 senses
chance%2:40:12:: wrong wordnet error
lemma 'bump' with part of speech 'v' has only 5 senses
bump%2:40:12:: wrong wordnet error
lemma 'happen' with

In [68]:
wn_concepts # TODO: can work better to align words in two sentences 

{(75, 0): {Synset('desire.v.03'),
  Synset('interpret.v.01'),
  Synset('never.r.01'),
  Synset('never.r.02'),
  Synset('school.n.07'),
  Synset('teacher.n.01'),
  Synset('teacher.n.02'),
  Synset('want.v.05')},
 (75, 1): {Synset('find.v.01'),
  Synset('rule.v.07'),
  Synset('school.n.07'),
  Synset('teacher.n.01'),
  Synset('teacher.n.02'),
  Synset('will.v.01')},
 (76, 0): {Synset('advertise.v.01'),
  Synset('advertise.v.02'),
  Synset('baseball_bat.n.01'),
  Synset('cut.v.13'),
  Synset('pizza.n.01'),
  Synset('publicize.v.01'),
  Synset('timbre.n.01')},
 (76, 1): {Synset('cut.v.06'),
  Synset('cut_back.v.01'),
  Synset('fell.v.01'),
  Synset('pizza.n.01'),
  Synset('reduce.v.01'),
  Synset('reduce.v.20'),
  Synset('tongue.n.03'),
  Synset('trim.v.09')},
 (77, 0): {Synset('build.v.10'),
  Synset('reach.v.07'),
  Synset('reconstruct.v.01'),
  Synset('swimming.n.01')},
 (77, 1): {Synset('build.v.10'),
  Synset('house.n.12'),
  Synset('reach.v.07'),
  Synset('reconstruct.v.01')},
 (78, 

In [69]:
wn_spe_concepts = {(idx, label):set() for (idx, label) in cached_spe_synsets.keys()}
for (idx, label), all_word_synsets in cached_spe_synsets.items(): # for each sentence
    for single_word_synsets in all_word_synsets: # for each synset of word
        # only keep the first wordnet words
        for synset in single_word_synsets['senses']:
            if synset['type'] == 'WordNetSense':
                sense_key = synset['properties']['senseKey']
                #lemma = wn.lemma_from_key(sense_key)
                #synsetkey = wn.synset(lemma)
                try:
                    synsetkey = wn.synset_from_sense_key(sense_key)
                    #print(synsetkey)
                    wn_spe_concepts[(idx, label)].add(synsetkey)
                except Exception as e:
                    print(e)
                    print(sense_key,'wrong wordnet error')
                

lemma 'happen' with part of speech 'v' has only 5 senses
happen%2:40:12:: wrong wordnet error
lemma 'chance' with part of speech 'v' has only 3 senses
chance%2:40:12:: wrong wordnet error
lemma 'bump' with part of speech 'v' has only 5 senses
bump%2:40:12:: wrong wordnet error
adjective satellite requested but only plain adjective found for lemma 'remote'
remote%5:00:00:unlikely:00 wrong wordnet error
lemma 'driving' with part of speech 'n' has only 2 senses
driving%1:04:03:: wrong wordnet error


In [70]:
wn_spe_concepts

{(75, 0): {Synset('interpret.v.01'),
  Synset('never.r.01'),
  Synset('never.r.02')},
 (75, 1): {Synset('detect.v.01'), Synset('meet.v.10')},
 (76, 0): {Synset('advertise.v.01'),
  Synset('advertise.v.02'),
  Synset('board.n.09'),
  Synset('publicize.v.01')},
 (76, 1): {Synset('tongue.n.03')},
 (77, 0): {Synset('swimming.n.01')},
 (77, 1): {Synset('house.n.12')},
 (78, 0): {Synset('build.v.10'),
  Synset('reach.v.07'),
  Synset('reconstruct.v.01')},
 (78, 1): {Synset('jewel.n.01'), Synset('muffin.n.01')},
 (79, 0): {Synset('chicken.n.01'), Synset('chicken.n.04')},
 (79, 1): {Synset('shuttlecock.n.01')},
 (80, 0): {Synset('cow.n.01')},
 (80, 1): {Synset('harmonica.n.01')},
 (81, 0): {Synset('alive.s.07'),
  Synset('even.r.01'),
  Synset('even.r.03'),
  Synset('give.v.44'),
  Synset('not.r.01'),
  Synset('still.r.04')},
 (81, 1): {Synset('pulse.n.03'),
  Synset('pulse.n.04'),
  Synset('rate.n.04'),
  Synset('then.r.01'),
  Synset('then.r.03'),
  Synset('therefore.r.01')},
 (82, 0): {Syns

In [34]:
import itertools

In [100]:
from sematch.semantic.similarity import WordNetSimilarity

wn_concept_perps = {(idx, label):0.0 for (idx, label) in cached_synsets.keys()}
wns = WordNetSimilarity()
for (idx, label) in wn_concepts.keys():
    counts = 0
    sum_sim = 0.0
    concepts = wn_concepts[(idx, label)]
    spe_concepts = wn_spe_concepts[(idx, label)]
    concepts = concepts.difference(spe_concepts)
    print(idx, label)
    print(concepts)
    print(spe_concepts)
    
    
    if len(concepts)>1 and len(spe_concepts)>=1:
        combs = list(itertools.product(concepts, spe_concepts))
        for syn_id_pair in combs:
            counts+=1
            print(syn_id_pair)
            #similarity = syn_id_pair[0].wup_similarity(syn_id_pair[1])
            #try:
            temp1 = [syn_id_pair[0]]
            temp2 = [syn_id_pair[1]]
            #similarity = wns.word_similarity_from_synset(temp1, temp2)
            #except Exception as e:
            #    print(e)
            #    similarity=0
            similarity = syn_id_pair[0].path_similarity(syn_id_pair[1])
            
            if similarity==0: # why none??
                sum_sim+=similarity
            elif similarity is None:
                print('wrong')
            else:
                sum_sim+=similarity
            
        print(sum_sim/counts)
        wn_concept_perps[(idx, label)] = sum_sim/counts
    else:
        #print(-99)
        print('no concept detected for this sentence')
        wn_concept_perps[(idx, label)] = -99
        pass

75 0
{Synset('desire.v.03'), Synset('want.v.05'), Synset('teacher.n.02'), Synset('school.n.07'), Synset('teacher.n.01')}
{Synset('never.r.01'), Synset('interpret.v.01'), Synset('never.r.02')}
(Synset('desire.v.03'), Synset('never.r.01'))
(Synset('desire.v.03'), Synset('interpret.v.01'))
(Synset('desire.v.03'), Synset('never.r.02'))
(Synset('want.v.05'), Synset('never.r.01'))
(Synset('want.v.05'), Synset('interpret.v.01'))
(Synset('want.v.05'), Synset('never.r.02'))
(Synset('teacher.n.02'), Synset('never.r.01'))
wrong
(Synset('teacher.n.02'), Synset('interpret.v.01'))
wrong
(Synset('teacher.n.02'), Synset('never.r.02'))
wrong
(Synset('school.n.07'), Synset('never.r.01'))
wrong
(Synset('school.n.07'), Synset('interpret.v.01'))
wrong
(Synset('school.n.07'), Synset('never.r.02'))
wrong
(Synset('teacher.n.01'), Synset('never.r.01'))
wrong
(Synset('teacher.n.01'), Synset('interpret.v.01'))
wrong
(Synset('teacher.n.01'), Synset('never.r.02'))
wrong
0.07074074074074074
75 1
{Synset('will.v.01'

wrong
(Synset('bicycle.n.01'), Synset('shoot.v.02'))
wrong
(Synset('bicycle.n.01'), Synset('hit.v.01'))
wrong
(Synset('foam.n.01'), Synset('shoot.v.01'))
wrong
(Synset('foam.n.01'), Synset('shoot.v.02'))
wrong
(Synset('foam.n.01'), Synset('hit.v.01'))
wrong
(Synset('foam.n.02'), Synset('shoot.v.01'))
wrong
(Synset('foam.n.02'), Synset('shoot.v.02'))
wrong
(Synset('foam.n.02'), Synset('hit.v.01'))
wrong
0.0
148 1
{Synset('play.v.03'), Synset('foam.n.02'), Synset('foam.n.01'), Synset('bicycle.n.01')}
{Synset('cinch.n.01'), Synset('play.n.06')}
(Synset('play.v.03'), Synset('cinch.n.01'))
(Synset('play.v.03'), Synset('play.n.06'))
(Synset('foam.n.02'), Synset('cinch.n.01'))
(Synset('foam.n.02'), Synset('play.n.06'))
(Synset('foam.n.01'), Synset('cinch.n.01'))
(Synset('foam.n.01'), Synset('play.n.06'))
(Synset('bicycle.n.01'), Synset('cinch.n.01'))
(Synset('bicycle.n.01'), Synset('play.n.06'))
0.06988728183581125
149 0
{Synset('today.n.01'), Synset('desire.v.03'), Synset('hour.n.04'), Synse

In [50]:
train_data.loc[train_data.id==76]

,id,special,whole,label,masked
173,76,a knife,she cut a pizza with a knife,1,she cut a pizza with [MASK]
174,76,an advertising board,she cut a pizza with an advertising board,0,she cut a pizza with [MASK]


In [94]:
wn_concept_perps

{(75, 0): 0.07074074074074074,
 (75, 1): 0.08002946127946127,
 (76, 0): 0.0433554292929293,
 (76, 1): 0.08111888111888112,
 (77, 0): 0.06547619047619048,
 (77, 1): 0.07539682539682539,
 (78, 0): 0.030902777777777776,
 (78, 1): 0.07000661375661375,
 (79, 0): 0.068367889420521,
 (79, 1): 0.05892857142857143,
 (80, 0): 0.006211180124223602,
 (80, 1): 0.008928571428571428,
 (81, 0): 0.14708994708994708,
 (81, 1): 0.029349007474007472,
 (82, 0): 0.011513157894736841,
 (82, 1): 0.0,
 (83, 0): -99,
 (83, 1): 0.1,
 (84, 0): 0.07645906724854093,
 (84, 1): -99,
 (85, 0): 0.0,
 (85, 1): 0.020227076077639985,
 (86, 0): 0.08358770858770859,
 (86, 1): 0.06683802308802309,
 (87, 0): 0.08838383838383838,
 (87, 1): 0.08333333333333333,
 (88, 0): 0.08701631701631704,
 (88, 1): 0.0775030525030525,
 (89, 0): 0.07586805555555555,
 (89, 1): 0.029852988953866146,
 (90, 0): 0.041666666666666664,
 (90, 1): 0.05,
 (91, 0): 0.06914402442477309,
 (91, 1): 0.07376789876789877,
 (92, 0): 0.04863163483072985,
 (92, 

In [101]:
predictions = []
for idx in samples['id'].unique():
    pos = wn_concept_perps[(idx,1)]
    neg = wn_concept_perps[(idx,0)]
    if pos<=0 or neg<=0:
        print('N/A')
        predictions.append('na')
    elif pos>neg:
        print('hit')
        predictions.append('hit')
    else:
        print('no hit')
        predictions.append('no hit')


hit
hit
hit
hit
no hit
hit
no hit
N/A
N/A
N/A
N/A
no hit
N/A
no hit
no hit
hit
hit
hit
N/A
N/A
N/A
hit
N/A
no hit
N/A
hit
no hit
hit
N/A
no hit
N/A
N/A
N/A
no hit
hit
N/A
N/A
hit
N/A
N/A
hit
no hit
hit
hit
hit
no hit
hit
no hit
no hit
hit
hit
no hit
N/A
hit
no hit
no hit
no hit
N/A
N/A
hit
no hit
N/A
N/A
no hit
N/A
hit
hit
N/A
N/A
no hit
hit
no hit
hit
N/A
N/A


## Results for the first 75 sentence pairs

In [102]:
pd.DataFrame(predictions,columns=['res'])['res'].value_counts()

na        28
hit       26
no hit    21
Name: res, dtype: int64

In [39]:
scores = []
for idx in samples['id'].unique():
    pos = wn_concept_perps[(idx,1)]
    neg = wn_concept_perps[(idx,0)]
    if pos<=0 or neg<=0:
        print('N/A')
        predictions.append('na')
    elif pos<neg:
        print('hit')
        predictions.append('hit')
    else:
        print('no hit')
        predictions.append('no hit')

no hit
hit
N/A
hit
hit
N/A
N/A
N/A
no hit
hit
N/A
hit
N/A
hit
hit
no hit
hit
hit
no hit
N/A
hit
N/A
hit
hit
N/A
no hit
hit
N/A
no hit
no hit
hit
no hit
hit
no hit
no hit
no hit
hit
no hit
N/A
hit
hit
no hit
no hit
N/A
no hit
hit
hit
hit
no hit
hit
no hit
hit
N/A
N/A
hit
no hit
N/A
N/A
no hit
N/A
hit
N/A
hit
no hit
N/A
hit
no hit
N/A
no hit
no hit
hit
hit
N/A
no hit
no hit
